In [1]:
import pandas as pd              
import numpy as np 
import tensorflow as tf

data = pd.read_csv("bible_data_set (with count and testament).csv")
data

,citation,book,chapter,verse,text,length (charcters),words,testament
0,Genesis 1:1,Genesis,1,1,In the beginning God created the heaven and th...,56,10,old
1,Genesis 1:2,Genesis,1,2,"And the earth was without form, and void; and ...",144,29,old
2,Genesis 1:3,Genesis,1,3,"And God said, Let there be light: and there wa...",56,11,old
3,Genesis 1:4,Genesis,1,4,"And God saw the light, that it was good: and G...",87,17,old
4,Genesis 1:5,Genesis,1,5,"And God called the light Day, and the darkness...",117,22,old
...,...,...,...,...,...,...,...,...
31097,Revelation 22:17,Revelation,22,17,"And the Spirit and the bride say, Come. And le...",170,33,new
31098,Revelation 22:18,Revelation,22,18,For I testify unto every man that heareth the ...,189,37,new
31099,Revelation 22:19,Revelation,22,19,And if any man shall take away from the words ...,209,44,new
31100,Revelation 22:20,Revelation,22,20,"He which testifieth these things saith, Surely...",97,16,new


In [2]:
import nltk
from nltk.stem import PorterStemmer

ps = PorterStemmer() 

vocabulary_size = 0
word2location = {}

def prepare_vocabulary(data):
    index = 0
    for sentance in data['text']:
        #sentance = sentance.lower()
        words = nltk.word_tokenize(sentance)
        for word in words:
            stemed_word = ps.stem(word)
            if stemed_word not in word2location:
                word2location[stemed_word] = index
                index += 1
    return index

def convert2vec(sentance):
    #sentance = sentance.lower()
    res_vec = np.zeros(vocabulary_size)
    words = nltk.word_tokenize(sentance)
    for word in words:
        stemed_word = ps.stem(word)
        if stemed_word in word2location:
            res_vec[word2location[stemed_word]]+=1
    return res_vec

vocabulary_size = prepare_vocabulary(data)
print("the size of the vocabulary is: ", vocabulary_size)
word2location

the size of the vocabulary is:  9421


{'In': 0,
 'the': 1,
 'begin': 2,
 'god': 3,
 'creat': 4,
 'heaven': 5,
 'and': 6,
 'earth': 7,
 '.': 8,
 'wa': 9,
 'without': 10,
 'form': 11,
 ',': 12,
 'void': 13,
 ';': 14,
 'dark': 15,
 'upon': 16,
 'face': 17,
 'of': 18,
 'deep': 19,
 'spirit': 20,
 'move': 21,
 'water': 22,
 'said': 23,
 'let': 24,
 'there': 25,
 'be': 26,
 'light': 27,
 ':': 28,
 'saw': 29,
 'that': 30,
 'it': 31,
 'good': 32,
 'divid': 33,
 'from': 34,
 'call': 35,
 'day': 36,
 'he': 37,
 'night': 38,
 'even': 39,
 'morn': 40,
 'were': 41,
 'first': 42,
 'a': 43,
 'firmament': 44,
 'in': 45,
 'midst': 46,
 'made': 47,
 'which': 48,
 'under': 49,
 'abov': 50,
 'so': 51,
 'second': 52,
 'gather': 53,
 'togeth': 54,
 'unto': 55,
 'one': 56,
 'place': 57,
 'dri': 58,
 'land': 59,
 'appear': 60,
 'sea': 61,
 'bring': 62,
 'forth': 63,
 'grass': 64,
 'herb': 65,
 'yield': 66,
 'seed': 67,
 'fruit': 68,
 'tree': 69,
 'after': 70,
 'hi': 71,
 'kind': 72,
 'whose': 73,
 'is': 74,
 'itself': 75,
 'brought': 76,
 'third'

In [14]:
tf.compat.v1.disable_v2_behavior()

books = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges',
            'Ruth', '1 Samuel', '2 Samuel', '1 Kings', '2 Kings', '1 Chronicles', '2 Chronicles',
            'Ezra', 'Nehemiah', 'Esther', 'Job', 'Psalms', 'Proverbs', 'Ecclesiastes',
            'Song of Solomon', 'Isaiah', 'Jeremiah', 'Lamentations', 'Ezekiel', 'Daniel',
            'Hosea', 'Joel', 'Amos', 'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk',
            'Zephaniah', 'Haggai',    'Zechariah',    'Malachi', 'Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans', '1 Corinthians',
            '2 Corinthians', 'Galatians', 'Ephesians', 'Philippians', 'Colossians',
            '1 Thessalonians', '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon',
            'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude',
            'Revelation']

def encode(line):
    res_vec = np.zeros(66)
    idx = books.index(data.iloc[line]['book'])
    res_vec[idx] = 1
    return res_vec

def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

features = vocabulary_size
categories = 66 #number of books
epsilon = 1e-12
x = tf.compat.v1.placeholder(tf.float32,[None,features])
y_ = tf.compat.v1.placeholder(tf.float32,[None,categories])
W = tf.compat.v1.Variable(tf.zeros([features,categories]))
b = tf.compat.v1.Variable(tf.zeros([categories]))

y = 1/(1.0 + tf.exp(-(tf.nn.sigmoid(tf.matmul(x,W) + b))))
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y)

update = tf.compat.v1.train.GradientDescentOptimizer(0.00001).minimize(cross_entropy)

data_x = np.array([convert2vec(data.iloc[i]['text']) for i in range(len(data['text']))])
data_y = np.array([encode(i) for i in range(len(data['text']))])

sess = tf.compat.v1.Session()

sess.run(tf.compat.v1.global_variables_initializer())

display_step = 2
#cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*categories)

for i in range(100):
    sess.run(update, feed_dict = {x:data_x, y_:data_y})
    if i % 1 == 0 :
        print('Iteration:' , i)
    
#y_values = tf.add(tf.matmul(x, W), b)
#y = tf.nn.softmax(y_values) 

test1 = "In the beginning God created the heaven and the earth. "
logistic_fun(np.matmul(np.array([convert2vec(test1)]),sess.run(W)) + sess.run(b))

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
It

array([[0.07649466, 0.07631566, 0.07500366, 0.07670446, 0.07487579,
        0.07396012, 0.07387909, 0.07245997, 0.07415788, 0.07414406,
        0.07444622, 0.07399125, 0.07529315, 0.07471229, 0.07311812,
        0.07331916, 0.07289043, 0.07730524, 0.08600156, 0.07737324,
        0.07308966, 0.07262475, 0.07553648, 0.07549468, 0.07296798,
        0.07571454, 0.07326525, 0.07286024, 0.07259599, 0.0727686 ,
        0.07240647, 0.07248765, 0.07262988, 0.07250454, 0.07255234,
        0.0725022 , 0.07245067, 0.07289188, 0.07247875, 0.07511473,
        0.0740763 , 0.0749725 , 0.0745134 , 0.07518714, 0.07381538,
        0.0736894 , 0.07285578, 0.07287622, 0.07296204, 0.07264226,
        0.07265077, 0.07257986, 0.07246933, 0.07263396, 0.07257561,
        0.07243888, 0.07237623, 0.07329824, 0.07264055, 0.07261919,
        0.07249518, 0.07267263, 0.07238292, 0.07239393, 0.07239532,
        0.07359288]])

this model does not fit to our problem because the outcome is binary and we have 66 categories therefor we should use softmax if we want logistic regression to succeed.
we could solve the testimant problem here but we already did that with linear regression.